<a href="https://colab.research.google.com/github/wolffg777/ait_datascience/blob/main/MusicRec01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO:
WORD2VEC KERAS CLASSIFICATION PROBLEM
- train/test split
- word2vec on descriptors and genres
  - change word2vec to import existing general word2vec model (search Google Code (googleNews-vectors-negative300.bin.gz))
  - first just working
  - then working out OOV (out of vocabulary) problems #fastText maybe ask chatgpt
- testing it to see improvement
- implementing dates (as continuous)
- easier things first
- spotify data integration


-brainstorm an idea for the name of the independent study on askbanner (limited to 30 characters)




check - train on domain text
notcheck - pretrained w2v
notcheck - use pretrained with domain adaptation  



In [48]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [49]:
from google.colab import files
files.upload()
!pip install kaggle
!chmod 600 /root/.kaggle/kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!kaggle datasets download -d tobennao/rym-top-5000
!unzip rym-top-5000.zip
!pip install spotipy

Saving kaggle.json to kaggle.json
n
rym-top-5000.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  rym-top-5000.zip
replace rym_clean1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace rym_raw1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [86]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [87]:
df = pd.read_csv('rym_clean1.csv')
# df.head(1)

# Remove rating_count, review_count, and avg_rating columns
df = df.drop(columns=['avg_rating'])
df = df.drop(columns=['rating_count'])
df = df.drop(columns=['review_count'])
df = df.drop(columns=['Unnamed: 0'])
df = df.drop(columns=['position'])
df = df.drop(columns=['release_type'])
df.head(5)

,release_name,artist_name,release_date,primary_genres,secondary_genres,descriptors
0,OK Computer,Radiohead,1997-06-16,"Alternative Rock, Art Rock",NaN,"melancholic, anxious, futuristic, malevocals, ..."
1,Kid A,Radiohead,2000-10-03,"Art Rock, Experimental Rock, Electronic","Ambient, Electronic, IDM","cold, melancholic, futuristic, anxious, atmosp..."
2,The Dark Side of the Moon,Pink Floyd,1973-03-23,"Art Rock, Progressive Rock","Psychedelic Rock, Space Rock","philosophical, atmospheric, introspective, exi..."
3,Loveless,My Bloody Valentine,1991-11-11,"Shoegaze, Noise Pop","Dream Pop, Neo-Psychedelia","noisy, ethereal, atmospheric, romantic, love, ..."
4,My Beautiful Dark Twisted Fantasy,Kanye West,2010-11-22,"Pop Rap, Hip Hop",Art Pop,"epic, boastful, passionate, sampling, hedonist..."


In [88]:
df = df.drop(columns=['artist_name'])
df = df.drop(columns=['primary_genres'])
df = df.drop(columns=['secondary_genres'])
df = df.drop(columns=['release_date'])

In [89]:
df.head(5)

,release_name,descriptors
0,OK Computer,"melancholic, anxious, futuristic, malevocals, ..."
1,Kid A,"cold, melancholic, futuristic, anxious, atmosp..."
2,The Dark Side of the Moon,"philosophical, atmospheric, introspective, exi..."
3,Loveless,"noisy, ethereal, atmospheric, romantic, love, ..."
4,My Beautiful Dark Twisted Fantasy,"epic, boastful, passionate, sampling, hedonist..."


In [58]:
from gensim.models import KeyedVectors

# Load the word2vec model
model = KeyedVectors.load_word2vec_format(path, binary=True)

In [90]:
#reformat descriptors properly
df['descriptors'] = df['descriptors'].str.split(', ')
desc_words = df['descriptors']

In [91]:
from keras.preprocessing.text import Tokenizer

# Assuming df['descriptors'] is a column with descriptors in string format (like "happy melodic soft")
texts = df['descriptors'].tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [92]:
print(sequences)

[[6, 24, 57, 1, 92, 98, 4, 31, 60, 89], [60, 6, 57, 24, 4, 27, 62, 68, 10, 1], [73, 4, 10, 92, 16, 44, 1, 17, 29, 43], [28, 38, 4, 25, 15, 32, 35, 17, 12, 18], [43, 55, 7, 40, 72, 86, 47, 2, 1, 10], [18, 1, 10, 6, 8, 16, 4, 13, 38, 58], [6, 4, 29, 1, 44, 116, 10, 58, 98, 8], [7, 30, 61, 62, 33, 8, 71, 2, 1, 15], [6, 31, 2, 24, 10, 1, 8, 92, 98, 58], [42, 50, 30, 44, 113, 10, 19, 1, 21, 7], [45, 36, 23, 19, 28, 1, 21, 12, 66, 106], [85, 43, 29, 34, 73, 33, 30, 1, 6, 68], [2, 13, 1, 49, 8, 26, 15, 25, 21, 48], [17, 9, 2, 1, 93, 18, 21, 26, 13, 41], [3, 46, 39, 1, 122, 56, 98, 7, 24, 124], [17, 2, 1, 21, 45, 9, 13, 41, 90, 33], [19, 88, 44, 50, 10, 1, 7, 138, 77, 8], [95, 2, 1, 47, 44, 7, 140, 8, 3, 102], [27, 11, 60, 24, 10, 31, 4, 74, 92, 89], [86, 22, 63, 55, 40, 28, 39, 3, 1, 36], [24, 27, 6, 4, 62, 68, 54, 60, 1, 11], [22, 63, 3, 39, 28, 106, 45, 23, 88, 40], [40, 9, 53, 62, 68, 54, 21, 1, 33, 55], [61, 7, 6, 2, 8, 91, 26, 30, 1, 44], [140, 69, 31, 44, 10, 24, 11, 43, 1, 98], [5, 24,

In [93]:
def descriptors_to_vector(descriptors):
    vectors = [model[word] for word in descriptors if word in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Apply the function to the dataframe
df['vector'] = df['descriptors'].apply(descriptors_to_vector)

In [94]:
padded_descriptors = pad_sequences(df['vector'].tolist(), maxlen=10, dtype='float32')
df.head(5)

,release_name,descriptors,vector
0,OK Computer,"[melancholic, anxious, futuristic, malevocals,...","[0.19141303, 0.041851893, -0.038394503, 0.0876..."
1,Kid A,"[cold, melancholic, futuristic, anxious, atmos...","[0.19380018, 0.01829232, 0.0573154, 0.06511773..."
2,The Dark Side of the Moon,"[philosophical, atmospheric, introspective, ex...","[0.22916412, -0.013671875, 0.034643173, 0.2033..."
3,Loveless,"[noisy, ethereal, atmospheric, romantic, love,...","[0.11493259, -0.09087457, -0.014350044, 0.0724..."
4,My Beautiful Dark Twisted Fantasy,"[epic, boastful, passionate, sampling, hedonis...","[0.10206434, -0.054423016, 0.12530178, 0.09427..."


In [95]:
df['descriptors'].iloc[0]

['melancholic',
 'anxious',
 'futuristic',
 'malevocals',
 'existential',
 'alienation',
 'atmospheric',
 'lonely',
 'cold',
 'pessimistic']

In [96]:
def recommend(descriptors_list):
    input_vector = descriptors_to_vector(descriptors_list)

    # Calculate cosine similarities between the input vector and all album vectors
    similarities = df['vector'].apply(lambda x: np.dot(input_vector, x) / (np.linalg.norm(input_vector) * np.linalg.norm(x)))

    # Get the index of the most similar album
    index_of_max_similarity = similarities.idxmax()

    # Return the album name
    return df.iloc[index_of_max_similarity]['release_name']

In [97]:
# Example usage:
input_descriptors = ['melancholic',
 'anxious',
 'futuristic',
 'malevocals',
 'existential',
 'alienation',
 'atmospheric',
 'lonely',
 'cold',
 'pessimistic']
recommended_album = recommend(input_descriptors)
print(recommended_album)

OK Computer


In [102]:
embedding_dim = 300  # As the Word2Vec model you downloaded has 300 dimensions
vocab_size = len(tokenizer.word_index) + 1

# Create an embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():  # tokenizer should be the tokenizer you used to process your text
    if word in model:
        embedding_vector = model[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


In [106]:
encoder = LabelEncoder()
df['release_name_label'] = encoder.fit_transform(df['release_name'])

# # Prepare training data
X = np.array(padded_descriptors)
y = df['release_name_label'].values

In [107]:
from sklearn.model_selection import train_test_split

# Assuming X is your feature matrix (padded_descriptors) and y is your target labels
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)  # using 20% of data for validation


In [108]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

modelSeq = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=padded_descriptors.shape[1],
              weights=[embedding_matrix], trainable=False),  # Using pre-trained embeddings and setting them non-trainable
    LSTM(128, return_sequences=True),  # Return sequences if you want to stack LSTMs
    Dropout(0.2),  # Add dropout for regularization
    LSTM(64),  # Another LSTM layer
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(len(df['release_name'].unique()), activation='softmax')
])

modelSeq.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Assuming you've created a validation set called X_val and y_val
history = modelSeq.fit(X, y, epochs=25, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stop, model_checkpoint])


Epoch 1/25
78/79 [============================>.] - ETA: 0s - loss: 8.5187 - accuracy: 0.0000e+00
Epoch 1: val_loss improved from inf to 8.50420, saving model to best_model.h5
79/79 [==============================] - 11s 80ms/step - loss: 8.5188 - accuracy: 0.0000e+00 - val_loss: 8.5042 - val_accuracy: 0.0000e+00
Epoch 2/25
78/79 [============================>.] - ETA: 0s - loss: 8.5080 - accuracy: 2.0032e-04
Epoch 2: val_loss improved from 8.50420 to 8.50416, saving model to best_model.h5
79/79 [==============================] - 6s 75ms/step - loss: 8.5080 - accuracy: 2.0000e-04 - val_loss: 8.5042 - val_accuracy: 0.0010
Epoch 3/25
78/79 [============================>.] - ETA: 0s - loss: 8.5079 - accuracy: 0.0000e+00
Epoch 3: val_loss improved from 8.50416 to 8.50411, saving model to best_model.h5
79/79 [==============================] - 5s 66ms/step - loss: 8.5079 - accuracy: 0.0000e+00 - val_loss: 8.5041 - val_accuracy: 0.0000e+00
Epoch 4/25
78/79 [============================>.] - E

things to do: stricter filtering of descriptors
one-hot the genres, word2vec (number of dimensions depends on embedding model, basic word2vec is 30 - 100), the descriptors and average the value and put it in space. tutorials on youtube seem good.

try to find a taxonomy of genres online, if not split up words and vaguely separating the musical words

Partial credit accuracy/reward system where "radiohead" for atmospheric even if the album recommended is not atmospheric

In [ ]:
import os
import base64
from requests import post, get
import json
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials


os.environ['SPOTIPY_CLIENT_ID'] = '08a0640020ff4e5d9713aaeacfbd1886'
os.environ['SPOTIPY_CLIENT_SECRET'] = '81cefe24f94b4f24af9fe1ad2e70ef4a'
os.environ['SPOTIPY_REDIRECT_URI'] = 'http://example.com/callback'

#Authorization Code Flow
scopes = "user-library-read user-top-read user-follow-read"

sp = spotipy.Spotify(
        auth_manager=spotipy.SpotifyOAuth(
          scope=scopes, open_browser=False))

In [ ]:
#Get Specified User's Playlists
playlists = sp.user_playlists('wolffg7')

while playlists:
    for i, playlist in enumerate(playlists['items']):
        #print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
        print("%4d %s" % (i + 1 + playlists['offset'], playlist['name']))

    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None


KeyboardInterrupt: ignored

In [ ]:
album_name = "OK Computer"
artist_name = "Radiohead"

# Search for the album
results = sp.search(q=f"album:{album_name} artist:{artist_name}", type="album")

album_id = results['albums']['items'][0]['id']

album_tracks = sp.album_tracks(album_id)

for track in album_tracks['items']:
    print(track['name'])


model architecture: pytorch simple neural network (feedforward neural network).

get a model working before break, even if its simpler (in terms of model architecture and pre-processing).

in recommendation stage, top n choices of different artists because otherwise it will just recommend songs from the same album
|
also figure out a way to vectorize/umbrella categories for genres
|
add column for frequency of each genre like
col 1 : space rock
col 2 : rock
col 3 : frequency 3
|
also separate the words in the descriptors (ie "malevocals" should be "male vocals")

still do this, but only useful for bringing in new information that is song specific, like lyrics and popularity

In [ ]:
# CODE GRAVEYARD

# # Flatten the lists in the "descriptors" column
# flattened_descriptors = df['descriptors'].explode()

# # Count the number of unique strings
# unique_count = flattened_descriptors.nunique()

# flattened_descriptors.head(5)

# # Use value_counts to count the frequency of elements in the column
# element_counts = flattened_descriptors.value_counts()

# # Print the result
# print(element_counts)



# # One-hot-encoding for DESCRIPTORS

# # Turn descriptors into a list of strings
# df['descriptors'] = df['descriptors'].str.split(',')

# descriptors_encoded = df['descriptors'].str.join('|').str.get_dummies()

# descriptor_frequencies = descriptors_encoded.iloc[:, 2:].sum()

# # print (descriptor_frequencies)

# rarity_threshold = 10

# # Clean column names by stripping spaces
# descriptors_encoded.columns = descriptors_encoded.columns.str.strip()

# rare_descriptors = descriptor_frequencies[descriptor_frequencies < rarity_threshold]

# # print(rare_descriptors)
# # print(len(rare_descriptors))